In [7]:
import pandas as pd
import statsmodels.api as sm
import pandas as pd 
import numpy as np 

# Load
df = pd.read_csv("historical_data.csv")

# One-hot encode categoricals
df = pd.get_dummies(df, columns=["province", "carcolour"], drop_first=True)
bool_cols = df.select_dtypes(include=["bool"]).columns
df[bool_cols] = df[bool_cols].astype(int)


# Response variable
y = df["claims"]

# Predictors (exclude 'id', 'claims', 'exp_years')
X = df.drop(columns=["id", "claims", "exp_years"])

# Add constant
X = sm.add_constant(X)

# Fit model with exposure offset
model = sm.GLM(
    y,
    X,
    family=sm.families.Poisson(),
    offset=np.log(df["exp_years"])
).fit()

print(model.summary())


                 Generalized Linear Model Regression Results                  
Dep. Variable:                 claims   No. Observations:                80000
Model:                            GLM   Df Residuals:                    79980
Model Family:                 Poisson   Df Model:                           19
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -32288.
Date:                Sat, 15 Nov 2025   Deviance:                       43565.
Time:                        19:24:43   Pearson chi2:                 8.09e+04
No. Iterations:                     6   Pseudo R-squ. (CS):            0.01824
Covariance Type:            nonrobust                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -1.4256      0.285  